In [1]:
!pip install lxml
!pip install selenium
!pip install cssselect
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'apt' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'cp' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
!pip install webdriver-manager

In [6]:
import pandas as pd
import numpy as np

from lxml import html
from lxml.cssselect import CSSSelector

# Create the web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\Arthur Lins\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache
<ipython-input-6-6d0708dffc36>:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
# Load the site on the web driver
url = "https://axie.zone/leaderboard"
driver.get(url)
driver.implicitly_wait(10)

# List to store links pointing to the player pages
players = []

try:
  # Find the anchor tags in the leaderboard table
  table_body_selector = ".leaderboard_table tbody"
  table_body = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, table_body_selector)))
  
  table_html = table_body[0].get_attribute('innerHTML')
  table_tree = html.fromstring(table_html)

  table_rows = table_tree.cssselect("tr")

  # Extract the href from the anchors found in the table
  for row in table_rows:
    players.append({
        "url": "https://axie.zone/" + row.cssselect("a")[0].get("href"),
        "winrate": int(row.cssselect("td:nth-child(6)")[0].text.replace("%", ""))
    })

  print("Retrieved", len(players), "URLs")
except TimeoutException:
  print("Element not properly loaded")

print(players)

Retrieved 101 URLs
[{'url': 'https://axie.zone//profile?ron_addr=0x62ea77d4a3e22490fc6a24cedd749f1f6b82ea28', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0xdaf19aef1ae54ed92d54997b6dd951d25ab94c94', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0x1b246e446336f55b4150294ccd39693fb4a8aa9b', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0x31ed9d2cc16b312cc4b3ec9030bff6522fc8da69', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0x9d7c8b278533f3533518f37d72b2d6fc013a36f9', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0x1ecb2d3ad316b04dd491916d1d8f428d008cb768', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0x1c4c23652b5e25d24bbd0195cd8bf43ab3bbec1f', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0xf52c5f320c7c8f2b62cbee4827aa69a4f921aa4b', 'winrate': 0}, {'url': 'https://axie.zone//profile?ron_addr=0xa2aa2a9b2f0409c4578994b052ab944051fab25d', 'winrate': 0}, {'url': 'https://axie.zone//profile

In [8]:
import re

# Used to extract the Axie parts from the user page
def get_parts(players):
  # List to store the scraped data
  axie_data = []

  # List for failed links
  failed = []

  # Counter that will be used to track the progress of the function
  progress = 0

  # Regex pattern to parse the parts
  p = re.compile('(.*): (.*) (\[.*\])')

  # Iterate through each of the player in the leaderboards
  for player in players:
    progress += 1
    try:
      # Load the player page on the driver
      driver.get(player["url"])
      driver.implicitly_wait(10)

      # Wait till the Axie info container loads
      _ = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container > div")))

      # Get the element that wraps the last used team container
      # This can be refactored to include the most used team of the player as well
      axie_container = WebDriverWait(driver, 0).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container")))

      # Convert the selenium webelement into a LXML tree
      axie_html = axie_container[0].get_attribute('innerHTML')
      axie_tree = html.fromstring(axie_html)

      # Get the Axie containers
      info_containers = axie_tree.cssselect(".search_result_wrapper")

      for info_container in info_containers:
        axie_parts = {}

        # Get the address of the owner
        axie_parts["User_Addr"] = player["url"].replace("https://axie.zone/profile?ron_addr=", "")

        # Get the Axie class
        axie_parts["Class"] = info_container.cssselect(".search_result")[0].get('class').replace("search_result ", "").capitalize()

        # Scrapes the parts from the Axie container
        parts = info_container.cssselect(".purity_part > div")

        for part in parts:
          m = p.match(part.get('title'))
          axie_parts[m.group(1)] = m.group(2)

        # Add winrate
        axie_parts["Winrate"] = player["winrate"]

        # Add the scraped parts to the main list of data
        axie_data.append(axie_parts)

      # To keep track and now that our script is working, we would print a message
      # containing a summary of the scraping's progress
      print("(", progress, "/", len(players) ,") Retrieved")
    except Exception as e:
      # We might also encounter an error where our collection script fails,
      # in the rare case where this would happen we would need to manually
      # revisit what went wrong and fix it.
      failed.append(player)
      print("(", progress, "/", len(players) ,") Error occured while retrieving", player["url"])
      print("Error received:", e)

  if len(failed) != 0:
    print("Retrying failed links")
    axie_data += get_parts(failed)

  return axie_data
  
# Start the scrapping process
res = get_parts(players)

( 1 / 101 ) Retrieved
( 2 / 101 ) Retrieved
( 3 / 101 ) Retrieved
( 4 / 101 ) Retrieved
( 5 / 101 ) Retrieved
( 6 / 101 ) Retrieved
( 7 / 101 ) Retrieved
( 8 / 101 ) Retrieved
( 9 / 101 ) Retrieved
( 10 / 101 ) Retrieved
( 11 / 101 ) Retrieved
( 12 / 101 ) Retrieved
( 13 / 101 ) Retrieved
( 14 / 101 ) Retrieved
( 15 / 101 ) Retrieved
( 16 / 101 ) Retrieved
( 17 / 101 ) Retrieved
( 18 / 101 ) Retrieved
( 19 / 101 ) Retrieved
( 20 / 101 ) Retrieved
( 21 / 101 ) Retrieved
( 22 / 101 ) Retrieved
( 23 / 101 ) Retrieved
( 24 / 101 ) Retrieved
( 25 / 101 ) Retrieved
( 26 / 101 ) Retrieved
( 27 / 101 ) Retrieved
( 28 / 101 ) Retrieved
( 29 / 101 ) Error occured while retrieving https://axie.zone//profile?ron_addr=0x939c65212b5c009968d8aa8c4e8e421df2dd742a
Error received: Message: 

( 30 / 101 ) Retrieved
( 31 / 101 ) Retrieved
( 32 / 101 ) Retrieved
( 33 / 101 ) Retrieved
( 34 / 101 ) Retrieved
( 35 / 101 ) Retrieved
( 36 / 101 ) Retrieved
( 37 / 101 ) Retrieved
( 38 / 101 ) Retrieved
( 39 / 1

In [9]:
# Convert the python list into a dataframe
df_res = pd.DataFrame(res)
df_res.head()

,User_Addr,Class,Eyes,Ears,Back,Mouth,Horn,Tail,Winrate
0,https://axie.zone//profile?ron_addr=0x62ea77d4...,Plant,Papi,Lotus,Bidens,Zigzag,Pliers,Yam,0
1,https://axie.zone//profile?ron_addr=0x62ea77d4...,Reptile,Confused,Friezard,Bone Sail,Toothless Bite,Cerastes,Grass Snake,0
2,https://axie.zone//profile?ron_addr=0x62ea77d4...,Dusk,Telescope,Seaslug,Indian Star,Tiny Turtle,Wing Horn,Grass Snake,0
3,https://axie.zone//profile?ron_addr=0xdaf19aef...,Plant,Confused,Clover,Pumpkin,Pincer,Leaf Bug,Yam,0
4,https://axie.zone//profile?ron_addr=0xdaf19aef...,Dusk,Gecko,Tiny Fan,Garish Worm,Toothless Bite,Cerastes,Grass Snake,0


In [10]:
# Export the dataframe into a csv file
df_res.to_csv("leaderboard.csv", index=False)